# Introduction

One of HiSparc's goals is to bring real science to high school classrooms. This document is part of that. Here you will find walkthrough of Python code that does callibration for HiSparc detectors. Along with explanation of bits of the code, this notebook also provides excersises to help you get used to Python and real data analysis.

If you follow this entire notebook you will have learned the following:
* Why Python is used in modern science.
* What Python packages are and what they do.
* How to read data files using Python.
* What binning is and why it's useful.
* How to make a simple graph using Python and Pylab.
* How to read and interpret your graphs.
* How to use the code in this notebook to check if your school's detectors are working well.

A lot of functions of the code in this notebook are beyond the scope of these teaching goals. To still try to satisfy the curious reader some additional reading will also be given so that you can try and figure out how it works for yourself. Perhaps in the future this notebook will expanded to also explicitly cover the more complicated aspects of the code.

Before we get started, this notebook will assume you already know some things. These subjects are listed below. If you're note familiar with one or more of them make sure you read or ask a teacher about them before you continue. 
* You are familiar with high school level mathematics.
* You know what HiSparc is and what research they do.

Now that all that is out of the way let's begin.

# Using Python

Python is a programming language often used by scientist to make scientific models or to analyse data. Perhaps you've already made a graph using Microsoft Excel before or even some basic modelling using programs like CMA's Coach. These programs are great for small data sets or smaller simulations, however when doing new science we often work with Terabytes of data, if not more. When working with so much data excel will simply crash, let alone let you manipulate the data or make graphs. 

To test this let's try to open the following file with excel: *LINK TO GITHUB*

This is slightly modified data file from HiSparc. You can use the link above to download it, it's about 200MB. Once you've done that open it with windows notepad (kladblok). It will open quite quickly, unfortunatly notepad has less graphing capability than an actual physical notepad. Don't close the notepad just yet. Let's open the file in excel as well first. To do this open excel and go to file -> open -> browse. Make sure to select "all files" (this can be found in the bottom right of the browsing window) in the browsing window. Now select the file you just opened. It will open a new window to let you import a .txt type file into excel.  Make sure to select "delimited" (this should be automatically selected for you) in this window and then press next. In the next window make sure to also select "other" and type "." (without the "", so just the period) in the text field next to other. Once you've done that, you can press finish. Excel will now freeze, this is normal after some time (you can try to count how long it takes if you wish) the file will open. 

It will probably give an error message telling you it could not load the entire file. We can check how much of the data it actually loaded by scrolling all the way down and checking the date (left most colomn). You can compare that to the latest date in notepad, they probably don't match. This is because excel just gives up trying to load more data after a while. Needless to say this is really bad if we want to actually use all the data.

Using python we can actually use the entire data set. So let's set that up here. This notebook has a very nice property, aside from displaying text it can also run python code like below. 

In [1]:
print("Hello World!")

Hello World!


The bit of code above is usually the first bit of code anyone trying to learn programming will write. It simply writes the text "Hello World!" in the output line. You can run the code by clicking on it to select the block of code and then pressing ctrl + enter or by pressing the play (run) button on the top of this window. When you do so the text "Hello World" will appear below the code. That area is called the output line. It is where any text, images or error messages will get displayed by the code.

Before we can begin programming ourselves I highly recommend you look up an online Python tutorial to learn more. Here are some examples:
* Example 1
* Example 2
* Example N

The first thing we will want to do is import some packages. Python can do a lot on it's own already but not everything. For example making a graph isn't possible without first writing an entire complicated program that can make them for you. This is where packages come in. One of the wonderful things about python is that many others have already written programs for many of the things you might want to do and then made those programs available for the public in packages. These packages can be installed and imported so that you can use the code other have already written. In the code block below we will import some packages for our program that will allow us to do more complicated math and make our own graphs easily.

In [2]:
#Import Packages
import time

import numpy as np

import pylab as pl

import tkinter.filedialog as tk

from scipy.optimize import curve_fit

Now that we have done that we can begin by importing data from HiSparc into our program in a way that we can use.

# Importing HiSparc Data

First you will need to download some real data from the HiSparc website here: https://data.hisparc.nl/data/download/.
Here you must select a station of your liking. We recommend 501: Nikhef or your own school if it has a detector with some recently collected data. For the date we recommend selecting a recent starting date and the day after it as an end date. You can select a larger timespan but this increases the size of your data file by a lot. This will make the download take longer as well as cause the program to take drastically longer to finish its computation. 

Next we need to get the information in the file you just downloaded into python. We'll be using the package "Tkinter" for this. Tkinter is a package that let's you import files into your code with a nice window like you're used to. First we need to set up tkinter. We do this by creating a "root" window. This is an empty window in which we can display things we want. But we don't want to display an empty window so we'll make it invisible right away as well. Then we can open a file selection window on top of the empty window. This window will then import the file(s) we selected into python, we will call them "files" to refer to later in the code. Last we need to close the window. To do all this we need the following code. Note that everything after the "#" simple doesn't add to the code itself but is instead a note describing what a line or some lines do.

If no window opens when you run the cell below, use "alt+tab" to cycle through all open windows. You will most likely find it there.

In [3]:
root = tk.Tk() #Create root window
root.withdraw() #Make the root window invisible
files = tk.askopenfilenames(parent=root, title='Choose any files') #Open a file explorer and import select files as "files"
root.destroy() #Close all windows

If this seems like a lot at once, that's okay because it is. You don't have to be able to do this yourself. The important take away is that we now have our file(s) imported into python and called them "files".  Notice that we could select more than one file. This is because "files" is actually list in which we can put multiple files. We need to keep that in mind in the future. 

We're not done yet however. Currently we have list with our files in it, however we want python to read what is in those files. Try to open the file you just downloaded with notepad to see what it looks like. It should look something like this:
![title](file_example.png)

First are a bunch of lines describing what is in the file. The third of those is the name of the station the data was taken from, the fifth the start and end date of the data in the file. Then follow some lines about the license for using the data. Lastly are some lines telling you what information is found in what column below.

After that we can see columns of numbers, which is the actual data. A nice way for python to read this would be as follows: 

-Make a list for each column of data and put those lists into a list of lists.

-Make the first "element" of that list the name of station so we can identify it later.

-Go through each line in and ignore it if its the text we don't care about.

-For each line with data, add the number in each column into the list that corresponds to that column.

When we do this for each line in the file we should end up with a list that contains a list for each column of data as well as the name of the station. This means that the first list in our list of lists will have the name of the station, the second list will have the date, the thirs list will have the time of the measurement etc. etc. You can read the file to check what each column is.

Below is the code that will do all this. Try to follow along with every step, if you don't understand what a bit of code does, you can always google it (for example you could google "Python append()" to figure out what append means).
When beginning to learn how to code don't be afraid to google a lot of things, even experienced programmers will continue to do this. It is a really good way to learn how to code. A lot of people will have had the same ideas and problems as you and other will have figured out how to do it, so you don't have to reinvent the wheel.

Keep in mind that in python we count starting form 0, not 1. So the first column is actually column number 0 while the second column is column number 1.

In [5]:
stations = [] #Make a list in which we can store the data of each file in a useful way, call that list stations

for file in files: #Loop through every file in the list files using the for loop, read as: "For each file in files do the following:"
    
    with open(file) as fa: #Open the file and call it "fa"

    #Read all lines
        station = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []] #Make a list of lists with a list for each column + 1 for the name of the station. We call this list of lists "station".
        for line_aa in fa.readlines(): #For every line (which we will call line_aa) in the file do the following

            if line_aa.startswith("# Station:"): #If the line starts with "# Station:" the do the following
                a = line_aa.split(':',2) #Split the line in half after the : and put both halves in a list called a, the second half will be the name of the station
                station[0].append(a[1]) #Add the second half the line (the name of the station) to the first list in station
            elif len(line_aa.split('\t',23))==23: #If the line doesn't start with "# Station:" then check if you can split it into 23 columns, if you can then do following
                a = line_aa.strip() #Remove all extra spaces, tabs and symbols
                cols = line_aa.split('\t',23) #Split the line at every tab. This will split all columns and put each number in a column in a list called cols

        #Now you have all the colums. Col1 through to Col4 countain data about the time of events. Col5 through to Col8 contain pulseheights of events. Col9 			through to 12 contain pulse integrals. Col 13 through to 16 contain the estimate for the number of particles detected. Col17 through to 20 contain the relative time between detections. Col 			21 contains trigger times. Col 22 and 23 the zenith and azimuth of the shower (currently not useable).
                for i, col in enumerate(cols): #For each column in the list col do the following, keep track of which column we're at with the variable i
                    if(i>3): #If it is the 5th column or above then do the following
                        station[i+1].append(float(col)) #Add the content of the column to the list of lists as a "float" (a number). Add it to the i+1th list in station (not the ith because the 1st list already has the name of station in it so we wanna skip that one) 
                    else: #If it's one of the first 4 columns then do the following
                        station[i+1].append(col) #Add the content of the column as a bit of text (a "string"). Add it to the i+1th list in station
            else: #If you the line doesn't start with "# Station:" and can't be split into 23 columns then ignore it
                continue
        stations.append(station.copy()) #Add a copy of the list of lists to list of stations (We're now three lists deep)
        station.clear() #Clear the contents of station so we can use it again for the next file


Okay that was a lot to go through, if you could follow all that, great job! If not, that's perfectly understandable. Try to go through it a few more times. If still don't understand what's going on try to ask a fellow student that does or perhaps a teacher. In the end the most important part to get is that we now have a list called stations. In that list we have a list, or multiple lists if we import mulitple files. This list (or lists) has the data of one station in it, let's therefor refer to each list in stations as a "station" for now. In each station are a number of lists (yes that's a lot of lists) the first list (list number 0 in python) contains the name of the station, every list after that contains data from each column in the file.

The next step is doing something useful with the data. Like making a graph of it!
# Making a Graph 



# Analyzing the Graph

# Comparing Stations